In [1]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()
Pkg.status()

  Activating project at `~/Documents/university /fall 2024/markov model/markvod_models/generated_data`


Status `~/Documents/university /fall 2024/markov model/markvod_models/generated_data/Project.toml`
⌃ [31c24e10] Distributions v0.25.119
⌃ [e9467ef8] GLMakie v0.11.3
  [429524aa] Optim v1.12.0
Info Packages marked with ⌃ have new versions available and may be upgradable.


In [2]:
include("dimer_difussion.jl")
include("free_difusion.jl")
include("state_generator.jl")

time_sequence_with_split (generic function with 1 method)

In [3]:

using GLMakie
function animate_particles(p1, p2, filename="particle_animation.mp4")
    n_steps = size(p1, 2)
    
    # Create figure
    fig = Figure(size=(600, 600))
    ax = Axis(fig[1, 1], aspect=DataAspect())
    
    # Calculate axis limits
    all_x = vcat(p1[1,:], p2[1,:])
    all_y = vcat(p1[2,:], p2[2,:])
    
    x_range = maximum(all_x) - minimum(all_x)
    y_range = maximum(all_y) - minimum(all_y)
    
    limits!(ax, -1, 1, -1, 1)
    
    # Create initial points
    point1 = scatter!([p1[1,1]], [p1[2,1]], color=:blue, markersize=10)
    point2 = scatter!([p2[1,1]], [p2[2,1]], color=:red, markersize=10)
    
    # Create animation
    record(fig, filename, 1:n_steps; framerate=10) do frame
        # Update particle positions for current frame
        point1[1] = [p1[1,frame]]
        point1[2] = [p1[2,frame]]
        
        point2[1] = [p2[1,frame]]
        point2[2] = [p2[2,frame]]
    end
    
    return filename
end




function reverse_columns_preserve_size(arr)
    num_rows, num_cols = size(arr)
    result = zeros(eltype(arr), num_rows, num_cols)
    
    for col in 1:num_cols
        result[:, col] = arr[:, num_cols - col + 1]
    end
    
    return result
end


reverse_columns_preserve_size (generic function with 1 method)

In [4]:
function shift(vec, shift_x, shift_y)
    return vec .+ [shift_x, shift_y]
end

shift (generic function with 1 method)

In [5]:
function path_correction(particle_path::Matrix{Float64}, new_destination::Vector{Float64})
    
    if size(particle_path, 1) != 2
        error("Particle path must be a 2×n matrix")
    end
    
    if length(new_destination) != 2
        error("New destination must be a 2-element vector")
    end
    
   
    n = size(particle_path, 2)
    
    
    start_point = particle_path[:, 1]
    original_end = particle_path[:, end]
    

    original_displacement = original_end - start_point
    desired_displacement = new_destination - start_point
    
  
    new_path = copy(particle_path)
    
    
    for i in 2:n
        
        t = (i - 1) / (n - 1)
        
       
        expected_position = start_point + original_displacement * t
        
       
        diffusion_component = particle_path[:, i] - expected_position
        
        
        new_expected_position = start_point + desired_displacement * t
        
       
        new_path[:, i] = new_expected_position + diffusion_component
    end
    
    return new_path
end

path_correction (generic function with 1 method)

# fucntion to generate the segements 

In [6]:
function segments(time_division::Vector, mode::Symbol;  D = 0.01, r = 0.2, box = 1.0, dt = 0.016)

if mode == :full && length(time_division) == 3 # all the sections 

        p1_middle, p2_middle = constrained_diffusion(steps = time_division[2],    D = D, r = r, box = box, dt = dt)

    init_post_r = ([p1_middle[1,end], p1_middle[2,end]],[p2_middle[1,end], p2_middle[2,end]])
    p1_right, p2_right = simulate_diffusion(initial_positions = init_post_r, steps = time_division[3],  D = D, box = box, dt = dt)

    init_post_l = ([p1_middle[1,1], p1_middle[2,1]], [p2_middle[1,1], p2_middle[2,1]])
    p1_left, p2_left = simulate_diffusion(initial_positions = init_post_l, steps = time_division[1], D = D, box = box, dt = dt)
    p1_left_reversed = reverse_columns_preserve_size(p1_left)
    p2_left_reversed = reverse_columns_preserve_size(p2_left)

    p1 = hcat(p1_left_reversed, p1_middle, p1_right)
    p2 = hcat(p2_left_reversed, p2_middle, p2_right)

    return p1, p2

elseif mode == :right_half && length(time_division) == 2 # middle and right half

    p1_middle, p2_middle = constrained_diffusion(steps = time_division[1],  D = D, r = r, box = box, dt = dt)

    init_post_r = ([p1_middle[1,end], p1_middle[2,end]],  [p2_middle[1,end], p2_middle[2,end]])
    p1_right, p2_right = simulate_diffusion(initial_positions = init_post_r, steps = time_division[2],  D = D, box = box, dt = dt)

    p1 = hcat(p1_middle, p1_right)
    p2 = hcat(p2_middle, p2_right)

    return p1, p2

elseif mode == :middle && length(time_division) == 1 # only middle half 
    p1_middle, p2_middle = constrained_diffusion(steps = time_division[1],  D = D, r = r, box = box, dt = dt)
    p1 = hcat(p1_middle)
    p2 = hcat(p2_middle)

    return p1, p2

elseif mode == :left_half && length(time_division) == 2 # middle and left half
    p1_middle, p2_middle = constrained_diffusion(steps = time_division[2],   D = D, r = r, box = box, dt = dt)

    init_post_l = ([p1_middle[1,1], p1_middle[2,1]],  [p2_middle[1,1], p2_middle[2,1]])
    p1_left, p2_left = simulate_diffusion(initial_positions = init_post_l,  steps = time_division[1], D = D, box = box, dt = dt)
    p1_left_reversed = reverse_columns_preserve_size(p1_left)
    p2_left_reversed = reverse_columns_preserve_size(p2_left)

    p1 = hcat(p1_left_reversed, p1_middle)
    p2 = hcat(p2_left_reversed, p2_middle)
    return p1, p2

else
    error("Invalid mode or time division")
end 
end

segments (generic function with 1 method)

In [7]:
p1,p2 = segments([10,40], :right_half, r = 0.01, box = 1.0, dt = 0.016)

([0.6803396563416071 0.682239270911576 … 0.6823699926672739 0.7044969070155868; 0.2416616101994781 0.2439042770221603 … 0.1831774760782321 0.16486210842823731], [0.6707867264741059 0.6807270390010035 … 0.7614728157047163 0.7749051340881934; 0.23870500218579024 0.2537892734636693 … 0.12176061131685137 0.1424584053859275])

# function to select the modes 

In [8]:
function modes_selection(section::Vector )

    size_section = length(section)
    if size_section == 3
        return :full
    elseif size_section == 2
        if section[1].first == 1 && section[2].first == 2
            return :left_half
        elseif section[1].first == 2 && section[2].first == 1
            return :right_half
        else
            error("Invalid section combination")
        end
    elseif size_section == 1
        return :middle
    else
        error("Invalid number of sections")
    end

end 

modes_selection (generic function with 1 method)

In [9]:
k12 = 0.5
k21 = 0.15 
states, steps = simulate_states(k12, k21, 10)
transitions = get_state_transitions(states)
time_in_state = time_sequence(states)
split= time_sequence_with_split(states)



5-element Vector{Any}:
 Any[1 => 7, 2 => 25, 1 => 2]
 Any[1 => 3, 2 => 56, 1 => 2]
 Any[1 => 2, 2 => 33, 1 => 14]
 Any[1 => 15, 2 => 36, 1 => 19]
 Any[1 => 20, 2 => 107, 1 => 1]

In [10]:
modes = modes_selection(split[3])

:full

# extracrt time stamps froms segements

In [11]:
function time_stamps(time_state)

    segments = []
    for i in 1:length(time_state)
        push!(segments, time_state[i].second)
    end


    return segments
end 

time_stamps (generic function with 1 method)

In [12]:
time = time_stamps(split[1])

3-element Vector{Any}:
  7
 25
  2

# function to modify array for path correction 

In [13]:
function path_correction!(particle_path, new_destination::Vector{Float64})
    if size(particle_path, 1) != 2
        error("Particle path must be a 2×n matrix")
    end
    
    if length(new_destination) != 2
        error("New destination must be a 2-element vector")
    end
    
    n = size(particle_path, 2)
    
    start_point = particle_path[:, 1]
    original_end = particle_path[:, end]
    
    original_displacement = original_end - start_point
    desired_displacement = new_destination - start_point
    
    for i in 2:n
        t = (i - 1) / (n - 1)
        
        expected_position = start_point + original_displacement * t
        
        diffusion_component = particle_path[:, i] - expected_position
        
        new_expected_position = start_point + desired_displacement * t
        
        particle_path[:, i] = new_expected_position + diffusion_component
    end
    
    return nothing
end

path_correction! (generic function with 1 method)

# function to genrate the whole simulation

In [14]:
function simulation(k_in ,k12_off,changes)

    k12 = k_in
    k21 = k12_off
    states, steps = simulate_states(k12, k21, changes)
    time_in_state = time_sequence_with_split(states)
    split = time_sequence(states)
    
    
    current_time  = 0 
    Particle_1 = []
    Particle_2 = []

    for i in 1:length(time_in_state)

        section_time_stamps = time_stamps(time_in_state[i])
        modes = modes_selection(section_time_stamps)
        println("section_time_stamps: ", section_time_stamps)
        println("modes: ", modes)
        if i == 1
            p1,p2= segments(section_time_stamps, modes, r = 0.01, box = 1.0, dt = 0.016)
            Particle_1 = p1
            Particle_2 = p2
            # println("p1: ", size(p1))
            # println("p2: ", size(p2))
        else 
            p1_temp, p2_temp = segments(section_time_stamps, modes, r = 0.01, box = 1.0, dt = 0.016)
            # println("p1_temp: ", size(p1_temp))
            # println("p2_temp: ", size(p2_temp))
            shift_x = -p1_temp[1,1] + Particle_1[1,end]
            shift_y = -p1_temp[2,1] + Particle_1[2,end]
            # println("shift_x: ", shift_x)
            # println("shift_y: ", shift_y)
            p1= shift(p1_temp, shift_x, shift_y)
            p2 = shift(p2_temp, shift_x, shift_y)
            # println("p1: ", size(p1[:,1]))
            # println("p2: ", size(p2[:,1]))
            # println("time_in_state: ", time_in_state[i])
            segment_view =  @view p2[:,1:time_in_state[i][1].second]
        
            path_correction!(segment_view, [Particle_2[1,end], Particle_2[2,end]])
            # println("part_1: ", size(Particle_1))
            # println("part_2: ", size(Particle_2))
            Particle_1 = hcat(Particle_1, p1)
            Particle_2 = hcat(Particle_2, p2)

        end


        
        curren_time = length(Particle_1) 

    end
   
    
    return Particle_1, Particle_2


end 

simulation (generic function with 1 method)

In [15]:
p1,p2 = simulation(0.5, 0.5, 10)

section_time_stamps: Any[16, 36, 24]
modes: full
section_time_stamps: Any[25, 35, 16]
modes: full
section_time_stamps: Any[17, 10, 4]
modes: full
section_time_stamps: Any[4, 2, 4]
modes: full
section_time_stamps: Any[5, 8, 1]
modes: full


([0.7136179801744497 0.6953081947864226 … 0.6931117228434044 0.6931117228434044; 0.0558714456190889 0.04347272025019474 … 0.39560124561407956 0.39560124561407956], [0.7742018404655968 0.7827486589702485 … 0.6987806011949808 0.6987806011949808; 0.2837398976032634 0.30661877502374146 … 0.38736329573349465 0.38736329573349465])

In [16]:

animate_particles(p1, p2, "total_animation_segments_1_2.mp4")



"total_animation_segments_1_2.mp4"

# implementation on the forward algorithm

In [17]:
function compute_free_density(particle1::Matrix{Float64}, particle2::Matrix{Float64}, index::Int; sigma=0.1, dt=0.01)


# Calculate relative positions for current frame
Δxn = particle1[1, index] - particle2[1, index]  
Δyn = particle1[2, index] - particle2[2, index]  

# Calculate relative positions for previous frame
Δxn_1 = particle1[1, index-1] - particle2[1, index-1]  
Δyn_1 = particle1[2, index-1] - particle2[2, index-1]  

# Calculate squared distances and distances
dn_1square = (Δxn_1^2) + (Δyn_1^2)
dn_1 = sqrt(dn_1square)
dn_square = (Δxn^2) + (Δyn^2)
dn = sqrt(dn_square)

# Calculate density using modified Bessel function
density_val = (dn/sigma^2) * (exp((-(dn_square) - (dn_1square))/sigma^2)) * modified_bessel(dt, dn, dn_1, sigma)

# Handle edge cases with numerical integration
if density_val == Inf || isnan(density_val) || density_val == 0
density_val = 0
for θ in 0:dt:2*pi
density_val += exp(-((dn_square) + (dn_1square) - (2*dn*dn_1*cos(θ)))/(2*sigma^2))
end 
density_val *= (dn/sigma^2)*dt
end

return density_val
end

compute_free_density (generic function with 1 method)

In [18]:
function forward_algorithm(particle1::Matrix{Float64}, particle2::Matrix{Float64}, d_dimer::Float64, param::Vector{Float64}; dt=0.01, sigma=0.1)

    N = min(size(particle1, 2), size(particle2, 2)) - 1
 
    alpha = zeros(2, N)
    scale = zeros(N)
    e1_vec =[]
    e2_vec = []
    # Ensure we don't go out of bounds when calculating initial state probabilities
    frame_index = min(2, N)
    

    e1_initial = compute_free_density(particle1, particle2, frame_index, sigma=sigma, dt=dt)
    
    e2_initial = compute_dimer_density(particle1, particle2, frame_index, d_dimer, sigma=sigma, dt=dt)
    push!(e1_vec, e1_initial)
    push!(e2_vec, e2_initial)
    alpha[1, 1] = e1_initial
    alpha[2, 1] = e2_initial
    
    scale[1] = sum(alpha[:, 1])
    alpha[:, 1] ./= max(scale[1], eps()) # Avoid division by zero
    
   
    k_on = param[1]
    k_off = param[2]
    T = [1-(k_on*dt) k_on*dt; 
         k_off*dt 1-(k_off*dt)]
    
    
    for t in 2:N
       
        frame_index = min(t + 2, size(particle1, 2))
        
        
        e1 = compute_free_density(particle1, particle2, frame_index, sigma=sigma, dt=dt)
        e2 = compute_dimer_density(particle1, particle2, frame_index, d_dimer, sigma=sigma, dt=dt)
        push!(e1_vec, e1)
        push!(e2_vec, e2)
        
        alpha[1, t] = (alpha[1, t-1]*T[1,1] + alpha[2, t-1]*T[2,1]) * e1
        alpha[2, t] = (alpha[1, t-1]*T[1,2] + alpha[2, t-1]*T[2,2]) * e2
        
        
        scale[t] = sum(alpha[:, t])
       # println("scale[t]: ", scale[t])
        alpha[:, t] ./= max(scale[t], eps()) 
    end
    
    
    loglikelihood = sum(log.(max.(scale, eps()))) 
    
    return alpha, loglikelihood, e1_vec, e2_vec
end

forward_algorithm (generic function with 1 method)

In [19]:
function compute_dimer_density(particle1::Matrix{Float64}, particle2::Matrix{Float64},index::Int, d_dimer::Float64; sigma=0.1, dt=0.01)

Δxn = particle1[1, index] - particle2[1, index]  
Δyn = particle1[2, index] - particle2[2, index]  


dn_square = (Δxn^2) + (Δyn^2)
dn = sqrt(dn_square)


density_val = (dn/sigma^2) * exp(((-(d_dimer^2) - (dn_square))/sigma^2)) * modified_bessel(dt, d_dimer, dn, sigma)


if density_val == Inf || isnan(density_val) || density_val == 0
density_val = 0
for θ in 0:dt:2*pi
density_val += exp(-((dn_square) + (d_dimer^2) - (2*dn*d_dimer*cos(θ)))/(2*sigma^2))
end 
density_val *= (dn/sigma^2)*dt
end

return density_val
end

compute_dimer_density (generic function with 1 method)

In [20]:
function modified_bessel(dt, d1, d2, σ)
    result_me = 0.0

    x = (d1*d2)/(σ^2)

    for θ in 0:dt:2*pi
        result_me += exp(x * cos(θ))
    end

    result_me  *= dt / (2*pi)

    return result_me 
end

modified_bessel (generic function with 1 method)

In [21]:
p1,p2 = simulation(0.5, 0.5, 10)

section_time_stamps: Any[17, 7, 9]
modes: full
section_time_stamps: Any[10, 13, 4]
modes: full
section_time_stamps: Any[5, 6, 34]
modes: full
section_time_stamps: Any[34, 32, 4]
modes: full
section_time_stamps: Any[5, 2, 1]
modes: full


([0.6431326165074658 0.6702830090017967 … 0.5094204843123884 0.5094204843123884; 0.6139994848703602 0.628030629542694 … 0.536388605767215 0.536388605767215], [0.4647559632967254 0.4612555271696165 … 0.5127122406918454 0.5127122406918454; 0.6045998624309844 0.5904925945207901 … 0.5458312928815011 0.5458312928815011])

In [32]:
p1

2×183 Matrix{Float64}:
 0.643133  0.670283  0.65081   0.649703  …  0.524271  0.50942   0.50942
 0.613999  0.628031  0.639241  0.625441     0.565686  0.536389  0.536389

In [33]:
σ = 0.1
p1_noise = p1 .+ σ*randn(size(p1))
p2_noise = p2 .+ σ*randn(size(p2))

2×183 Matrix{Float64}:
 0.348769  0.712421  0.599919  0.518253  …  0.418232  0.694868  0.612078
 0.549821  0.725054  0.522959  0.455865     0.574287  0.558512  0.299718

In [34]:
alpha, loglikelihood,e1,e2 = forward_algorithm(p1, p2, 0.01, [0.5, 0.5])
alpha_noise, loglikelihood_noise,e1_noise,e2_noise = forward_algorithm(p1_noise, p2_noise, 0.01, [0.5, 0.5])

([2.4435845085604526e-7 0.0005126087945746504 … 0.00012088590964076781 5.216463473263608e-5; 0.9999997556415492 0.9994873912054254 … 0.9998791140903593 0.9999478353652673], -227.2464703094831, Any[8.089603440865931e-7, 0.2418460834641489, 0.12167661447764713, 0.06184619140126461, 0.12120890346699296, 0.113404589674842, 0.42288954345480845, 0.02190418636710745, 0.008973725437117825, 0.09154845938907788  …  1.5970229087581818, 3.1956458913657304, 0.8400037736700086, 0.3515065027640296, 2.128068108112865, 0.5027349019173933, 0.16497549060862857, 0.008989432052709604, 0.0053610359167799325, 0.0053610359167799325], Any[3.310547040940546, 2.3697272720823483, 0.14428568201569425, 1.113439290055268, 0.2508450770628554, 2.4272158327284936, 0.8086068294993455, 0.023944206249134042, 4.224507865080351, 0.08146385206126512  …  3.389814757076643, 3.6879331326914353, 0.8893253622840737, 3.9951402285711763, 4.042472500081377, 0.609401549275654, 3.9565448829491485, 0.007194817453166303, 0.5288367357796

In [35]:
states_round =round.(alpha[1,:])
states_noise_round = round.(alpha_noise[1,:])

182-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [36]:
using CairoMakie
fig = Figure(size=(800, 600))
ax = Axis(fig[1, 1], title="Free density", xlabel="Time", ylabel="probaility")
ax1 = Axis(fig[2, 1], title="Dimer density", xlabel="Time", ylabel="probaility")
ax2 = Axis(fig[3, 1], title="position_diference in x ", xlabel="Time", ylabel="distacne")
ax3 = Axis(fig[4, 1], title="position_diference in y ", xlabel="Time", ylabel="distacne")
ax4 = Axis(fig[6, 1], title="alpha[2,:]", xlabel="Time", ylabel="dimer probaility")
ax5 = Axis(fig[5, 1], title="alpha[1,:]", xlabel="Time", ylabel="free probability")

lines!(ax, e1, color=:blue, label="Free density")
lines!(ax1, e2, color=:red, label="Dimer density")
lines!(ax2, p1[1,:] - p2[1,:], color=:green, label="x")
lines!(ax3, p1[2,:] - p2[2,:], color=:orange, label="y")
lines!(ax4, alpha[1,:], color=:purple, label="dimer")
lines!(ax5, alpha[2,:], color=:black, label="free")
axislegend(ax, position = :rt) 
display(fig)


GLMakie.Screen(...)

In [37]:
save("probability_fig.png", fig)

In [38]:
fig2 = Figure(size=(800, 600))
ax6 = Axis(fig2[1, 1], title="Free density with noise", xlabel="Time", ylabel="probaility")
ax7 = Axis(fig2[2, 1], title="Dimer density with noise", xlabel="Time", ylabel="probaility") 
ax8 = Axis(fig2[3, 1], title="alpha_noise[1,:]", xlabel="Time", ylabel="free probaility")
ax9 = Axis(fig2[4, 1], title="alpha_noise[2,:]", xlabel="Time", ylabel="dimer probaility")
lines!(ax6, e1_noise, color=:blue, label="Free density with noise")
lines!(ax7, e2_noise, color=:red, label="Dimer density with noise")
lines!(ax8, alpha_noise[1,:], color=:black, label="free with noise")
lines!(ax9, alpha_noise[2,:], color=:purple, label="dimer with noise")
axislegend(ax6, position = :rt)
display(fig2)   

GLMakie.Screen(...)

In [39]:
save("probability_noise.png", fig2)

In [42]:
using Optim

function objective_function(param, p1, p2, d_dimer)
    _, loglikelihood,e1,e2 = forward_algorithm(p1, p2, d_dimer, param)
    return -loglikelihood
end


initial_params = [0.1, 0.1]


function wrapper(param)
    return objective_function(param, p1, p2, 0.01)
end
function wrapper_noise(param)
    return objective_function(param, p1_noise, p2_noise, 0.01)
end

result = optimize(wrapper, initial_params, NelderMead())
result_noise = optimize(wrapper_noise, initial_params, NelderMead())



optimal_params = Optim.minimizer(result)
optimal_params_noise = Optim.minimizer(result_noise)
k_on_optimized = optimal_params[1]
k_off_optimized = optimal_params[2]
k_off_optimized_noise = optimal_params_noise[2]
k_on_optimized_noise = optimal_params_noise[1]


_, final_loglikelihood,e1,e2 = forward_algorithm(p1, p2, 0.01, optimal_params)
_, final_loglikelihood_noise,e1_noise,e2_noise = forward_algorithm(p1_noise, p2_noise, 0.01, optimal_params_noise)


([2.4435845085604526e-7 -0.11365582099120797 … -0.010241231671660802 -0.010241231671660802; 0.9999997556415492 1.113655820991208 … 1.010241231671661 1.010241231671661], 60627.90552741055, Any[8.089603440865931e-7, 0.2418460834641489, 0.12167661447764713, 0.06184619140126461, 0.12120890346699296, 0.113404589674842, 0.42288954345480845, 0.02190418636710745, 0.008973725437117825, 0.09154845938907788  …  1.5970229087581818, 3.1956458913657304, 0.8400037736700086, 0.3515065027640296, 2.128068108112865, 0.5027349019173933, 0.16497549060862857, 0.008989432052709604, 0.0053610359167799325, 0.0053610359167799325], Any[3.310547040940546, 2.3697272720823483, 0.14428568201569425, 1.113439290055268, 0.2508450770628554, 2.4272158327284936, 0.8086068294993455, 0.023944206249134042, 4.224507865080351, 0.08146385206126512  …  3.389814757076643, 3.6879331326914353, 0.8893253622840737, 3.9951402285711763, 4.042472500081377, 0.609401549275654, 3.9565448829491485, 0.007194817453166303, 0.5288367357796016, 

In [43]:
println("Optimized k_on: ", k_on_optimized)
println("Optimized k_off: ", k_off_optimized)
println("Final Log-Likelihood: ", final_loglikelihood)

println("Optimized k_on with noise: ", k_on_optimized_noise)
println("Optimized k_off with noise: ", k_off_optimized_noise)
println("Final Log-Likelihood with noise: ", final_loglikelihood_noise)

Optimized k_on: 2.8051410137668514e9
Optimized k_off: -8.531025784883523e9
Final Log-Likelihood: 3317.122428525262
Optimized k_on with noise: -7.380678393980563e146
Optimized k_off with noise: -9.890639624961675e147
Final Log-Likelihood with noise: 60627.90552741055


-1.0049548462044867e226